In [40]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [41]:
balance_data = pd.read_csv("Decision_Tree_ Dataset.csv")

if 'sum' in balance_data.columns:
    balance_data = balance_data.drop('sum', axis="columns")
else:
    print("Column sum does not exit")

balance_data.columns = ['Initial Payment', 'Last Payment', 'Credit Score', 'House Number', "Result"]
balance_data.head()


,Initial Payment,Last Payment,Credit Score,House Number,Result
0,201,10018,250,3046,yes
1,205,10016,395,3044,yes
2,257,10129,109,3251,yes
3,246,10064,324,3137,yes
4,117,10115,496,3094,yes


In [42]:
# Separate features (X) and target (Y)
X = balance_data.iloc[:, 0:4] 
Y = balance_data.iloc[:, 4]

# Encode the target variable ('yes' -> 1, 'no' -> 0)
le = LabelEncoder()
Y_encode = le.fit_transform(Y)

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_encode, test_size=0.2, random_state=42)

# Create and train the Decision Tree Classifier
clf = DecisionTreeRegressor(random_state=42)
clf.fit(X_train, Y_train)

# Make predictions
y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy Score:", accuracy*100)

# Detailed classification report
print("\nClassification Report:")
print(classification_report(Y_test, y_pred, target_names=le.classes_))

Accuracy Score: 0.93

Classification Report:
              precision    recall  f1-score   support

          No       0.96      0.91      0.93       109
         yes       0.90      0.96      0.93        91

    accuracy                           0.93       200
   macro avg       0.93      0.93      0.93       200
weighted avg       0.93      0.93      0.93       200

